In [1]:
import os
os.chdir('../')

In [2]:
from ultralytics import YOLO
from src.HowlerMonkey.utils.common import read_yaml, create_directories, save_json, get_latest_model, clean_scores

/Users/guido/Library/CloudStorage/OneDrive-PUCRS-BR/Mestrado/Dissertacao/Howler-Monkey/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
model = YOLO(get_latest_model('artifacts/training'))

In [4]:
model

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    model_path: Path
    data_path: Path
    all_params: dict
    mlflow_uri: str
    model_name: str

In [6]:
from src.HowlerMonkey.constants import *
from src.HowlerMonkey.utils.common import read_yaml, create_directories, save_json, get_latest_model, clean_scores


In [7]:
print(os.environ["MLFLOW_TRACKING_USERNAME"])

GMainardi


In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        evaluation_config = EvaluationConfig(
            model_path = self.config.training.root_dir,
            data_path  = self.config.training.data_file_path,
            all_params = self.params,
            mlflow_uri = os.environ["MLFLOW_TRACKING_URI"],
            model_name = self.config.training.model_path.split('/')[-1].split('.')[0]
        )
        return evaluation_config
    

In [9]:
import pathlib as Path
import mlflow
import mlflow.pytorch
from ultralytics import YOLO
from urllib.parse import urlparse

In [10]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    @staticmethod
    def load_model(model_path: Path):
        return YOLO(get_latest_model(model_path))


    def evaluation(self):

        self.model = self.load_model(self.config.model_path)
        self.score = self.model.val(data=self.config.data_path).results_dict
        self.save_score()

    def save_score(self):
        self.scores = clean_scores(self.score)
        save_json(path="scores.json", data=self.scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                self.scores
            )

            model = self.load_model(self.config.model_path)

            if tracking_url_type_store != "file":
                mlflow.pytorch.log_model(model, "model", registered_model_name=self.config.model_name)
            else:
                mlflow.pytorch.log_model(model, "model")



In [11]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2024-03-31 21:32:20,113: INFO: common] yaml file: config/config.yaml loaded successfully
[2024-03-31 21:32:20,114: INFO: common] yaml file: params.yaml loaded successfully
[2024-03-31 21:32:20,114: INFO: common] Creating directory: artifacts
Ultralytics YOLOv8.1.34 🚀 Python-3.9.6 torch-2.2.1 CPU (Apple M1 Pro)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /Users/guido/Library/CloudStorage/OneDrive-PUCRS-BR/Mestrado/Dissertacao/Howler-Monkey/artifacts/data_ingestion/data/labels/val.cache... 5 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5/5 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.82it/s]


                   all          5         10       0.92        0.5      0.995      0.647
                  body          5          5       0.84          1      0.995      0.697
                  face          5          5          1          0      0.995      0.597
Speed: 1.1ms preprocess, 48.3ms inference, 0.0ms loss, 6.6ms postprocess per image
Results saved to runs/detect/val16
[2024-03-31 21:32:23,056: INFO: common] json file saved at: scores.json


_SklearnCustomModelPicklingError: Pickling custom sklearn model YOLO failed when saving model: cannot pickle 'torch._C.Generator' object